In [1]:
import numpy as np
import copy

# Problem modelling imports
from docplex.mp.model import Model

# Qiskit imports
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp

In [2]:
def create_problem(mu: np.array, sigma: np.array, total: int = 4) -> QuadraticProgram:
    """Solve the quadratic program using docplex."""

    mdl = Model()
    x = [mdl.binary_var("x%s" % i) for i in range(len(sigma))]

    objective = mdl.sum([mu[i] * x[i] for i in range(len(mu))])
    objective += mdl.sum(
        [sigma[i, j] * x[i] * x[j] for i in range(len(mu)) for j in range(len(mu))]
    )
    mdl.minimize(objective)
    cost = mdl.sum(x)
    # mdl.add_constraint(cost == total)

    qp = from_docplex_mp(mdl)
    return qp


def relax_problem(problem) -> QuadraticProgram:
    """Change all variables to continuous."""
    relaxed_problem = copy.deepcopy(problem)
    for variable in relaxed_problem.variables:
        variable.vartype = VarType.CONTINUOUS

    return relaxed_problem

In [3]:
#clustering, 2heads, 4 elements
p=100

weight = np.array([1,0.5,0.7,0.04,1,0.2,0.9,0.5])

# 0<=w<=1

mu = np.array([-p]*8) - weight

sigma = np.array(
    [
        [0, 0, 0, 0, p, 0, 0, 0],
        [0, 0, 0, 0, 0, p, 0, 0],
        [0, 0, 0, 0, 0 ,0, p, 0],
        [0, 0, 0, 0, 0, 0, 0, p],
        [p, 0, 0, 0, 0, 0, 0, 0],
        [0, p, 0, 0, 0, 0, 0, 0],
        [0, 0, p, 0, 0, 0, 0, 0],
        [0, 0, 0, p, 0, 0, 0, 0],
    ]
)


In [4]:
qubo = create_problem(mu, sigma)
qp = relax_problem(QuadraticProgramToQubo().convert(qubo))

sol = CplexOptimizer().solve(qubo)
c_stars = sol.samples[0].x


qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 101 x0 - 100.500000000000 x1 - 100.700000000000 x2 - 100.040000000000 x3
      - 101 x4 - 100.200000000000 x5 - 100.900000000000 x6 - 100.500000000000 x7
      + [ 400 x0*x4 + 400 x1*x5 + 400 x2*x6 + 400 x3*x7 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1

Binaries
 x0 x1 x2 x3 x4 x5 x6 x7
End

In [6]:
# IBMQ.save_account('3ee456b1263b90639e8489ea13fa0d36e449c828416371c209e21b0f90c5da5b7b54bbfc39fd0eb826246e9099f7d98eab5b7c4b5c111ddb37353f3c4605512b', overwrite=True)
# IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q-education', group='ibm-quantum-1', project='quantum-hackatho')
# qaoa_mes = QAOAClient(
#     provider=provider, backend=provider.get_backend("ibmq_jakarta"), reps=2, alpha=0.75
# )

In [86]:
from qiskit_optimization.runtime import QAOAClient
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit import IBMQ

IBMQ.save_account('f0afd9f4ea051d12d9b01a92fbc12a7a9886adf6bf448699e806d95e733ea355273e7c008318caeaf4ca2fc91415da2d24eb7064360253a4755ad36cd8c7e926', overwrite=True)

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

qaoa_mes = QAOAClient(
    provider=provider, backend=provider.get_backend("ibmq_qasm_simulator"), reps=2, alpha=0.75
)

qaoa = MinimumEigenOptimizer(qaoa_mes)

qaoa.solve(qubo)

ibmqfactory.load_account:WARNING:2022-06-28 16:54:15,351: Credentials are already in use. The existing account in the session will be replaced.


optimal function value: -402.9
optimal value: [1. 1. 0. 0. 0. 0. 1. 1.]
status: SUCCESS

In [7]:
algorithm_globals.random_seed = 12345
quantum_instance = QuantumInstance(
    BasicAer.get_backend("statevector_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0.0, 1.0])
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)

qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: -402.9
optimal value: [1. 1. 0. 0. 0. 0. 1. 1.]
status: SUCCESS
